In [23]:
import pandas as pd
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
#import cdata.amazons3 as mod

import dash
import dash_bootstrap_components as dbc
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from sklearn.metrics import confusion_matrix, classification_report
import xgboost as xgb
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

In [24]:

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.MINTY])

main_df = pd.read_csv('history_final.csv')

main_df['day_of_week'] = pd.to_datetime(main_df['Date']).dt.dayofweek
main_df['month'] = pd.to_datetime(main_df['Date']).dt.month
main_df['Sector'] = main_df['Sector'].map(
    {'Industrials': 0, 'Health Care': 1, 'Information Technology': 2, 'Consumer Discretionary': 3, 'Financials': 4,
     'Materials': 5, 'Real Estate': 6, 'Consumer Staples': 7, 'Energy': 8, 'Utilities': 9,
     'Telecommunication Services': 10})
encoded_df = pd.read_csv('encoded_df.csv')

y = main_df['Label']

names = main_df['Symbol'].unique()

X_train, X_test, y_train, y_test = train_test_split(encoded_df, y, test_size=0.2, random_state=123)
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(X_train, y_train)

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("Next Day Stock Movement Prediction", className='text-center text-primary'))
    ]),
    dbc.Row([
        dbc.Col([dcc.Dropdown(id='slct-symbol',
                              options=names,
                              multi=False,
                              value='GOOG',
                              style={'width': '40%'}
                              ),
                 html.Div(id='output_container', children=[]),
                 html.Br(),
                 dcc.Graph(id='stock', figure={}),
                 html.Br()])
    ]),
    dbc.Row([
        dbc.Col([dcc.Dropdown(id='slct-ml',
                              options=['One', 'Two', 'Three', 'Four'],
                              multi=False,
                              style={'width': '40%'}),

                 #html.Div(id='output_container1', children=[]),
                 html.Br(),
                 dcc.Graph(id='ml1', figure={}),
                 html.Br()
                 ])
    ])
], fluid=False)
app.run(jupyter_mode="external")


#server = app.server

@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='stock', component_property='figure')],
    [Input(component_id='slct-symbol', component_property='value')]
)
def update_graph(option_slctd):
    print(option_slctd)
    print(type(option_slctd))

    container = "Stock chosen is: {}".format(option_slctd)

    dff = main_df.copy()
    dff = dff[dff['Symbol'] == option_slctd]

    dff['diff'] = dff['Close'] - dff['Open']
    dff.loc[dff['diff'] >= 0, 'color'] = 'green'
    dff.loc[dff['diff'] < 0, 'color'] = 'red'

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Candlestick(x=dff['Date'],
                                 open=dff['Open'],
                                 high=dff['High'],
                                 low=dff['Low'],
                                 close=dff['Close'],
                                 name='Price'))

    fig.update_layout(xaxis_rangeslider_visible=True)  # show range slider
    fig.update_layout(title={'text': option_slctd, 'x': 0.5})
    fig.update_xaxes(rangebreaks=[
        dict(bounds=['sat', 'mon']),  # hide weekends
    ])
    fig.layout.template = 'ggplot2'

    return container, fig


@app.callback(
    [Output(component_id='ml1', component_property='figure')],
    [Input(component_id='slct-ml', component_property='value'), ],
    prevent_initial_call=True
)
def update_figure(option_slctd):
    print(option_slctd)
    print(type(option_slctd))

    #container = "Model chosen is: {}".format(option_slctd)

    test_df = main_df[main_df['Symbol'] == 'GOOG']
    testy_df = test_df.drop(
        ['Label', 'Date', 'Open', 'Close', 'CloseDiff', 'Symbol', 'VolumeScaledNormalized', 'Low', 'High'], axis=1)

    tester = test_df['Label']

    pred = xgb_model.predict(testy_df)

    conf_matrix = confusion_matrix(tester, pred)

    labels = ['Down', 'Up']
    fig = ff.create_annotated_heatmap(
        z=conf_matrix,
        x=labels,
        y=labels,
        colorscale='Blues',
    )
    fig.update_layout(
        title='Confusion Matrix - Stock Price Movement Prediction',
        xaxis=dict(title='Predicted', side='bottom'),
        yaxis=dict(title='Actual'),
    )

    return fig


if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


AssertionError: The setup method 'errorhandler' can no longer be called on the application. It has already handled its first request, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the application are done before running it.

In [ ]:
#create train and test sets
from sklearn.model_selection import train_test_split

X = main_df[['Close', 'VolumeScaledNormalized', 'CloseDiff', 'WeekDiff']]
y = main_df['Actual Movement']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_test)
#run random forest classifier
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

#show confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)


In [ ]:
#write code for a kneighbors classifier
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

#show confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)


